In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/lower-back-pain-symptoms-dataset/Dataset_spine.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.drop("Unnamed: 13",axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()   # we find that there is no missing values in the dataset

In [ ]:
df['Class_att'].value_counts()

In [ ]:
df['Class_att'].replace({'Abnormal':1,"Normal":0},inplace=True)

In [ ]:
df['Class_att'].value_counts()

In [ ]:
df.corr()

In [ ]:
X=df.drop('Class_att',axis=1)
y=df['Class_att']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from statsmodels.tools import add_constant as add_constant

In [ ]:
df_constant=add_constant(df)
df_constant

In [ ]:
from statsmodels.tools import add_constant as add_constant
import statsmodels.api as sm
cols=df_constant.columns[:-1]
model=sm.Logit(df.Class_att,df_constant[cols])
result=model.fit()
result.summary()

In [ ]:
largest=round(result.pvalues,3).nlargest(12)
largest

In [ ]:
def back_elimnation_feature(data_frame,dep_var,col_list):
    while len(col_list) > 0:
        model=sm.Logit(dep_var,data_frame[col_list])
        result=model.fit(disp=0)
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.05):
            return result
            break
        else:
            col_list=col_list.drop(largest_pvalue.index)

result=back_elimnation_feature(df_constant,df.Class_att,cols)

In [ ]:
result.summary()

In [ ]:
X_final=df[['Col2','Col4','Col5','Col6']]
Y_final=df['Class_att']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_final,Y_final,random_state=10,test_size=0.3)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train,y_train)
y_predict=model.predict(x_test)

In [ ]:
from sklearn import metrics

In [ ]:
acc=metrics.accuracy_score(y_test,y_predict)*100
acc

In [ ]:
cm=metrics.confusion_matrix(y_test,y_predict)
cm

In [ ]:
y_test.value_counts()

In [ ]:
tpr=cm[1,1]/cm[1,:].sum()
print("The sensitivity of the model is:",tpr)

In [ ]:
tnr=cm[0,0]/cm[0,:].sum()
print("The specificity of the model is:",tnr)